In [ ]:
import streamlit as st
import numpy as np
import pandas as pd

st.set_page_config(page_title="Bernoulli Frequency Demo", layout="centered")
st.title("Bernoulli Frequency Demo")

st.markdown(
    "Pick the success probability \\(p\\). The app simulates Bernoulli trials for "
    "several sample sizes \\(n\\) and shows the empirical frequency and how far it is "
    "from \\(p\\) (absolute difference)."
)

# --- User controls ---
p = st.slider("Choose p", min_value=0.0, max_value=1.0, value=0.5, step=0.01)
seed = st.number_input("Random seed (optional)", value=42, step=1)
include_big_n = st.checkbox("Include n = 10,000,000 (may take a while)", value=False)

base_ns = [10, 50, 100, 1000]
n_values = base_ns + ([10_000_000] if include_big_n else [])

run = st.button("Run simulation")

# --- Computation ---
if run:
    rng = np.random.default_rng(int(seed))

    rows = []
    for n in n_values:
        successes = rng.binomial(n=n, p=p)  # draw number of successes without large arrays
        result = successes / n
        abs_diff = abs(result - p)
        rows.append({"n": n, "result": result, "|result - p|": abs_diff})

    df = pd.DataFrame(rows)

    st.subheader("Results")
    st.dataframe(
        df.style.format({"result": "{:.4f}", "|result - p|": "{:.4f}"}),
        use_container_width=True,
    )

    st.subheader("Visualization")
    st.line_chart(
        df.set_index("n")[["result"]],
        x_label="n",
        y_label="result",
    )
    st.line_chart(
        df.set_index("n")[["|result - p|"]],
        x_label="n",
        y_label="|result - p|",
    )

    st.caption(
        "The theoretical value is p. As n grows, the empirical frequency typically "
        "approaches p and the absolute deviation |result - p| tends to decrease."
    )
else:
    st.info("Set parameters and click **Run simulation**.")